# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [10]:
import dlt
import duckdb

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

## Question 1

In [3]:
limit = 5
generator = square_root_generator(limit)

count = 0
for sqrt_value in generator:
    count += sqrt_value

print(count)

8.382332347441762


## Question 2

In [37]:
limit = 13
generator = square_root_generator(limit)

count = 0
for sqrt_value in generator:
    pass

print(sqrt_value)

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Question 3

In [7]:
pipeline = dlt.pipeline(pipeline_name="people_random_data",
						destination='duckdb', 
						dataset_name='people_data')

In [29]:
info = pipeline.run(people_1, 
                    table_name="people_data_age", 
                    write_disposition="replace")

print(info)

Pipeline people_random_data load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////Users/starlord1711/Documents/BigData/DataEngPractice/DataEngZoomCamp/workshops/dlt/people_random_data.duckdb location to store data
Load package 1707907899.581955 is LOADED and contains no failed jobs


In [30]:
info = pipeline.run(people_2, 
                    table_name="people_data_age", 
                    write_disposition="append")

print(info)

Pipeline people_random_data load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////Users/starlord1711/Documents/BigData/DataEngPractice/DataEngZoomCamp/workshops/dlt/people_random_data.duckdb location to store data
Load package 1707907901.7111602 is LOADED and contains no failed jobs


In [23]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

In [27]:
people_data_age_data = conn.sql("SELECT * FROM people_data.people_data_age")
display(people_data_age_data)

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707907584.7188249 │ kM818W6f7+goAg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707907584.7188249 │ 1vLCXDpjKXs9lw │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707907584.7188249 │ iUMkpOBiAhxc9Q │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707907584.7188249 │ m8iZScoh1Tg67A │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707907584.7188249 │ zQXZ4yJo09EFag │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1707907609.653485  │ R+I2idHG7bbUvw │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1707907609.653485  │ sRe6ZVXXl0EiRQ │ Job_4      │
│     5 │ 

In [25]:
people_data_age_sum = conn.sql("SELECT SUM(age) FROM people_data.people_data_age")
display(people_data_age_sum)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [31]:
info = pipeline.run(people_1, 
                    table_name="people_data_age_merged", 
                    write_disposition="replace")

print(info)

Pipeline people_random_data load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////Users/starlord1711/Documents/BigData/DataEngPractice/DataEngZoomCamp/workshops/dlt/people_random_data.duckdb location to store data
Load package 1707907909.005261 is LOADED and contains no failed jobs


In [32]:
info = pipeline.run(people_2, 
                    table_name="people_data_age_merged", 
                    write_disposition="merge",
                    primary_key="ID")

print(info)

Pipeline people_random_data load step completed in 0.41 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:////Users/starlord1711/Documents/BigData/DataEngPractice/DataEngZoomCamp/workshops/dlt/people_random_data.duckdb location to store data
Load package 1707907912.3463612 is LOADED and contains no failed jobs


In [34]:
people_data_age_data = conn.sql("SELECT * FROM people_data.people_data_age_merged ORDER by id")
display(people_data_age_data)

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707907909.005261  │ PIfWF4iuYIYX9Q │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707907909.005261  │ 7q2vk2MqYWu4OA │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1707907912.3463612 │ ghbXR6nxYnNlOg │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1707907912.3463612 │ eP/mMwEnSNc6SQ │ Job_4      │
│     5 │ Person_5 │    35 │ City_B  │ 1707907912.3463612 │ ZPnj3eu83Hc4FQ │ Job_5      │
│     6 │ Person_6 │    36 │ City_B  │ 1707907912.3463612 │ 63XORUvLs89weA │ Job_6      │
│     7 │ Person_7 │    37 │ City_B  │ 1707907912.3463612 │ V+ZNHo9h3qpD/A │ Job_7      │
│     8 │ 

In [35]:
people_data_age_sum = conn.sql("SELECT SUM(age) FROM people_data.people_data_age_merged")
display(people_data_age_sum)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘